In [1]:
import pandas as pd
import networkx as nx


In [2]:
relationship_df = pd.read_csv("./data/relationship_df.csv")

# Create a graph from a pandas dataframe
G = nx.from_pandas_edgelist(relationship_df, 
                            source = "source", 
                            target = "target", 
                            edge_attr = "value", 
                            create_using = nx.Graph())


In [3]:
from algorithmx import jupyter_canvas
canvas = jupyter_canvas()
canvas.nodes(G.nodes).add()
canvas.edges(G.edges).add()
canvas.edges(G.edges).add(
    labels=lambda e: {0: {'text': G.edges[e]['value']}}
)

canvas

JupyterWidget(events=['{"attrs": {"nodes": {"Agnes": {}, "T\\u00f3ti": {}, "Natan": {}, "P\\u00e9tur": {}, "J\…

## Clustering coefficient

#### $C_i = \frac{2E_i}{(k_i(k_i - 1))}$

In [34]:
def cc(G):
    # input: graph, target
    # output: C, clustering coefficient of all nodes
    
    values = {}

    for node in G.nodes:
        neighbors = []
        E = []
        for neighbor in G.neighbors(node):
            neighbors.append(neighbor)

        for i in neighbors:
            for j in neighbors:
                if i > j: # python auto converts char names to ASCII integers, allowing for unique identifiers
                    edge = (i,j) # this means duplicate edges (e.g. a-b, b-a) will always be detected
                elif i == j:
                    break
                else:
                    edge = (j,i)
                if edge in G.edges:
                    if edge not in E:
                        E.append(edge)
        E = len(E)
        k = len(neighbors)
        C = (2*E)/(k*(k-1))

        values[node] = C

    return values
    

cc(G)

{'Agnes': 0.2537878787878788,
 'Tóti': 0.31339031339031337,
 'Natan': 0.3433333333333333,
 'Pétur': 0.8888888888888888,
 'Jón': 0.477124183006536,
 'Margrét': 0.39920948616600793,
 'Steina': 0.6666666666666666,
 'Lauga': 0.6923076923076923,
 'Kristín': 0.8666666666666667,
 'Ingibjörg': 0.5,
 'Jóas': 0.3888888888888889,
 'Inga': 0.6071428571428571,
 'Páll': 0.9,
 'Róslín': 0.7272727272727273,
 'Fridrik': 0.4666666666666667,
 'Gudmundur': 0.7818181818181819,
 'Björn': 0.4642857142857143,
 'Sigga': 0.44166666666666665,
 'Thorvardur': 0.6666666666666666,
 'Sigrídur': 1.0,
 'Haukur': 1.0,
 'Gudrún': 0.5333333333333333,
 'Dagga': 1.0,
 'Bjarni': 1.0,
 'Helga': 0.6666666666666666,
 'Magnús': 0.8,
 'Ingveldur': 0.6666666666666666,
 'Kjartan': 0.3055555555555556,
 'Gudbjörg': 0.7333333333333333,
 'Ragnar': 0.0,
 'Rósa': 0.5555555555555556,
 'Karitas': 0.75,
 'Worm': 0.8333333333333334,
 'Daníel': 1.0,
 'María': 0.6666666666666666,
 'Thórunn': 1.0,
 'Thóranna': 1.0,
 'Thórbjörg': 1.0,
 'Ospak': 

Converting clustering coefficient values to colours (indicating cliques)

In [5]:
import colorsys

def create_color_spectrum(value):
    # Define the minimum and maximum colors in RGB
    min_color_rgb = (173, 208, 255)  # Light blue
    max_color_rgb = (255, 109, 49)  # Orange

    # Calculate the range of colors between the minimum and maximum colors
    color_range_rgb = (
        max_color_rgb[0] - min_color_rgb[0],
        max_color_rgb[1] - min_color_rgb[1],
        max_color_rgb[2] - min_color_rgb[2],
    )

    # Adjust the input value to start at 0 instead of 0.2, and scale it to the range between 0 and 1
    adjusted_value = (value - 0.2) / 0.8

    # Calculate the RGB color for the given input value
    r = int(min_color_rgb[0] + adjusted_value * color_range_rgb[0])
    g = int(min_color_rgb[1] + adjusted_value * color_range_rgb[1])
    b = int(min_color_rgb[2] + adjusted_value * color_range_rgb[2])

    # Return the RGB color as a hex string
    return f"{'#{:02x}{:02x}{:02x}'.format(r, g, b)}"

# Example usage
cliques = cc(G)
node_warmth = {}

for node in cliques:
    node_warmth[node] = create_color_spectrum(cliques[node])

print(node_warmth)

{'Agnes': '#b2c9f1', 'Tóti': '#b8c1e1', 'Natan': '#bbbeda', 'Pétur': '#f37a4d', 'Jón': '#c9adb7', 'Margrét': '#c1b7cb', 'Steina': '#dc9686', 'Lauga': '#df9380', 'Kristín': '#f17d53', 'Ingibjörg': '#cbaab1', 'Jóas': '#c0b8ce', 'Inga': '#d69d96', 'Páll': '#f4794a', 'Róslín': '#e38e77', 'Fridrik': '#c8afba', 'Gudmundur': '#e88869', 'Björn': '#c8afba', 'Sigga': '#c5b2c0', 'Thorvardur': '#dc9686', 'Sigrídur': '#ff6d31', 'Haukur': '#ff6d31', 'Gudrún': '#cfa6a9', 'Dagga': '#ff6d31', 'Bjarni': '#ff6d31', 'Helga': '#dc9686', 'Magnús': '#ea8564', 'Ingveldur': '#dc9686', 'Kjartan': '#b7c2e3', 'Gudbjörg': '#e38e75', 'Ragnar': '#98e8132', 'Rósa': '#d1a4a3', 'Karitas': '#e58b71', 'Worm': '#ed815b', 'Daníel': '#ff6d31', 'María': '#dc9686', 'Thórunn': '#ff6d31', 'Thóranna': '#ff6d31', 'Thórbjörg': '#ff6d31', 'Ospak': '#ff6d31'}


## PageRank

#### $ Pr(A) = \frac{1-d}{N} + d(\frac{Pr(B)}{L(B)} + \frac{Pr(C)}{L(C)} + ...)$

In [6]:
def pagerank(G, d=0.15, N=len(G.nodes)):
    # input: G, graph
    # output: pagerank of all nodes

    rankings = {}

    # initially set their rankings evenly
    for node in G.nodes:
        rankings[node] = 1/N
    
    for x in range(100): # 100 iterations is probably enough
        for node in G.nodes:
            total = 0
            for neighbor in G.neighbors(node):
                total += rankings[neighbor]/len(list(G.neighbors(neighbor)))

            rankings[node] = (1-d)/N + d * total
    
    # return sum(rankings.values())
    return rankings

pagerank(G)

{'Agnes': 0.0394738472481509,
 'Tóti': 0.03505468743895345,
 'Natan': 0.03409864004285598,
 'Pétur': 0.024025542585117257,
 'Jón': 0.028460546242910965,
 'Margrét': 0.031788523220386736,
 'Steina': 0.025683920517197322,
 'Lauga': 0.026094396115482442,
 'Kristín': 0.024678816412522626,
 'Ingibjörg': 0.02559770301057355,
 'Jóas': 0.026617840766010474,
 'Inga': 0.024628022857808237,
 'Páll': 0.022997302549032674,
 'Róslín': 0.025271257775407705,
 'Fridrik': 0.028538779118864675,
 'Gudmundur': 0.02503045440239164,
 'Björn': 0.024902100065575552,
 'Sigga': 0.029475348890835404,
 'Thorvardur': 0.025056946041268034,
 'Sigrídur': 0.022740609400569843,
 'Haukur': 0.02252655461644871,
 'Gudrún': 0.025421803794402776,
 'Dagga': 0.023009183571939425,
 'Bjarni': 0.023635973698355815,
 'Helga': 0.023092634247703983,
 'Magnús': 0.023126179930798234,
 'Ingveldur': 0.022925775861538687,
 'Kjartan': 0.028076632556974963,
 'Gudbjörg': 0.023986934515303236,
 'Ragnar': 0.02244224224922206,
 'Rósa': 0.02527

## Diameter

In [7]:
def diameter(G):
    # input: G, graph
    # output: Average shortest path in G

    edgelist = {}

    for i in G.nodes:
        for j in G.nodes:
            if G.has_edge(i, j):
                edgelist[f"({i},{j})"] = G[i][j]['value']
            elif i == j:
                edgelist[f"({i},{j})"] = 0
            else:
                edgelist[f"({i},{j})"] = float('inf')

    for k in G.nodes:
        for i in G.nodes:
            for j in G.nodes:
                if edgelist[f"({i},{j})"] > edgelist[f"({i},{k})"] + edgelist[f"({k},{j})"]:
                    edgelist[f"({i},{j})"] = edgelist[f"({i},{k})"] + edgelist[f"({k},{j})"]
    
    mean = sum(edgelist.values())/(len(G.edges)*2)
    diameter = max(edgelist.values())

    return f"mean shortest path is {mean} diameter is {diameter}"

diameter(G)

'mean shortest path is 45.44864864864865 diameter is 27'

## Degree distribution

$ P_{deg}(K) $ = fraction of nodes in the graph with degree $k$

In [8]:
def dd(G):
    # input: G, graph
    # output: fraction of nodes in G with degree k

    distrib = {}
    for node in G.nodes:
        degree = G.degree(node)
        if degree in distrib:
            distrib[degree] += 1
        else:
            distrib[degree] = 1

    for node in distrib:
        distrib[node] = f"{distrib[node]}/{len(G.nodes)}"

    return distrib

dd(G)


{33: '1/39',
 27: '1/39',
 25: '1/39',
 9: '5/39',
 18: '1/39',
 23: '1/39',
 13: '2/39',
 10: '1/39',
 8: '4/39',
 5: '2/39',
 11: '2/39',
 15: '1/39',
 16: '1/39',
 7: '2/39',
 4: '6/39',
 2: '3/39',
 6: '3/39',
 3: '2/39'}

## Results

In [9]:
print(f"clustering coefficient values {cc(G)}")
print(diameter(G))
print(f"pagerank values {pagerank(G)}")
print(f"degree distribution {dd(G)}")



clustering coefficient values {'Agnes': 0.2537878787878788, 'Tóti': 0.31339031339031337, 'Natan': 0.3433333333333333, 'Pétur': 0.8888888888888888, 'Jón': 0.477124183006536, 'Margrét': 0.39920948616600793, 'Steina': 0.6666666666666666, 'Lauga': 0.6923076923076923, 'Kristín': 0.8666666666666667, 'Ingibjörg': 0.5, 'Jóas': 0.3888888888888889, 'Inga': 0.6071428571428571, 'Páll': 0.9, 'Róslín': 0.7272727272727273, 'Fridrik': 0.4666666666666667, 'Gudmundur': 0.7818181818181819, 'Björn': 0.4642857142857143, 'Sigga': 0.44166666666666665, 'Thorvardur': 0.6666666666666666, 'Sigrídur': 1.0, 'Haukur': 1.0, 'Gudrún': 0.5333333333333333, 'Dagga': 1.0, 'Bjarni': 1.0, 'Helga': 0.6666666666666666, 'Magnús': 0.8, 'Ingveldur': 0.6666666666666666, 'Kjartan': 0.3055555555555556, 'Gudbjörg': 0.7333333333333333, 'Ragnar': 0.0, 'Rósa': 0.5555555555555556, 'Karitas': 0.75, 'Worm': 0.8333333333333334, 'Daníel': 1.0, 'María': 0.6666666666666666, 'Thórunn': 1.0, 'Thóranna': 1.0, 'Thórbjörg': 1.0, 'Ospak': 1.0}
mea

visualisation

In [10]:
from pyvis.network import Network
net = Network(notebook = True, width="1920px", height="1080px", bgcolor='#222222', font_color='white')

# Identify hubs with degree distribution and make hub nodes larger
node_degree = dict(G.degree)

# Identify cliques with clustering coefficient and colour them warmer
cliques = cc(G)
node_warmth = {}

for node in cliques:
    node_warmth[node] = create_color_spectrum(cliques[node])

# Setting up node size and colour attribute
nx.set_node_attributes(G, node_degree, 'size')
nx.set_node_attributes(G, node_warmth, 'color')

net.from_nx(G)
net.show("./data/br.html")

./data/br.html
